In [1]:
import os

import mlflow
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## About data set
This data set contains information about load applicants
We will solve a binary classification problem to determine if their loan will be approved
Note: The point of this project is to explore MLflow, so I will not be putting too much effort into EDA

In [2]:
loan_df = pd.read_csv("data/loan_train.csv")
loan_df

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area,Status
0,Male,No,0,Graduate,No,584900,0.0,15000000,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,458300,150800.0,12800000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,300000,0.0,6600000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,258300,235800.0,12000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,600000,0.0,14100000,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,290000,0.0,7100000,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,410600,0.0,4000000,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,807200,24000.0,25300000,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,758300,0.0,18700000,360.0,1.0,Urban,Y


## EDA

In [3]:
loan_df.shape

(614, 12)

In [4]:
loan_df.describe()

,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History
count,6.140000e+02,6.140000e+02,6.140000e+02,600.00000,564.000000
mean,5.403459e+05,1.621246e+05,1.414104e+07,342.00000,0.842199
std,6.109042e+05,2.926248e+05,8.815682e+06,65.12041,0.364878
min,1.500000e+04,0.000000e+00,0.000000e+00,12.00000,0.000000
25%,2.877500e+05,0.000000e+00,9.800000e+06,360.00000,1.000000
50%,3.812500e+05,1.188500e+05,1.250000e+07,360.00000,1.000000
75%,5.795000e+05,2.297250e+05,1.647500e+07,360.00000,1.000000
max,8.100000e+06,4.166700e+06,7.000000e+07,480.00000,1.000000


In [5]:
loan_df.isnull().sum()

Gender                13
Married                3
Dependents            15
Education              0
Self_Employed         32
Applicant_Income       0
Coapplicant_Income     0
Loan_Amount            0
Term                  14
Credit_History        50
Area                   0
Status                 0
dtype: int64

### Data Preparation

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

y = loan_df[["Status"]]
X = loan_df.drop(["Status"], axis=1)

# I want to use KNN for the categorical variables. This can not be done directly
# First label encode the categories while keeping NaNs
X = X.apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))

X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [7]:
from sklearn.impute import KNNImputer

# We will perform KNN imputation to fill in our data. Why? Quick and easy for out purposes

knn_imputer = KNNImputer()


X_train_tr = knn_imputer.fit_transform(X_train)

X_test_tr = knn_imputer.transform(X_test)





### MLflow

Logging a logistic regression model

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
with mlflow.start_run():
    lr = LogisticRegression()
    lr.fit(X_train_tr, y_train)

    mlflow.sklearn.log_model(lr, "logistic regression model")
    y_pred = lr.predict(X_test_tr)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("model_accuracy", accuracy)

C:\Users\eric1\anaconda3\envs\mlops\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\eric1\anaconda3\envs\mlops\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\eric1\anaconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Now I can view the mlflow UI on my local host by running "mlflow ui" in the terminal